<h1> Energy Analyzer </h1>

<h3> Francesco Cristoforo Conti, Gabriele Sanguedolce </h3>

A.A. 2021/2022

<a href="https://www.docker.com/">Energy Analyzer</a>
















<h3>Introduzione</h3>

Lo scopo del seguente progetto è quello di acquisire dati, inerenti le produzioni energetiche e le emissioni di Co2 da parte dei principali paesi del continente europeo, da una sorgente real-time simulata, elaborare tali dati ottenendo delle previsioni su specifici valori ed una serie di analisi statistiche per poi visualizzare il tutto su una dashboard interattiva di Kibana.

<h3>Pipeline</h3>

<img src="images/pipeline.png" width="800px">


<br>
<h3><img src="images/electricitymap.png" width="20px" align="left">  &nbsp; Dataset locale  </h3>


Per simulare una sorgente dati in real-time, sono stati raccolti venti giorni di dati dal sito <a href="https://app.electricitymaps.com/map">Electricity Map</a>.
Dal momento che per alcune nazioni i dati non erano disponibili, si è optato per acquisire dati soltanto dalle seguenti nazioni: ES, PT, FR, IT, CH, AT, SI, HR, ME, GR, BE, NL, DE, GB, IE, IS, DK, NO, SE, FI, RU, PL, EE, LV, LT, CZ, SK, HU, RS, BG, RO, TR, GE.
I dati trattati, come detto in precedenza, riguardano la produzione di energia dalle diverse fonti energetiche e la quantità di Co2 (gCO₂eq/kWh) emessa, un esempio è il seguente:

``` json
{
    "_isFinestGranularity": true,
    "co2intensity": 266.89,
    "countryCode": "DE",
    "maxProduction": 21428.75,
    "price": {
        "currency": "EUR",
        "value": 201.99
    },
    "production": {
        "biomass": 4710.75,
        "coal": 17383.5,
        "gas": 3639.5,
        "geothermal": 21.0,
        "hydro": 1768.5,
        "nuclear": 3996.5,
        "oil": 8.0,
        "solar": 13485.0,
        "unknown": 346.0,
        "wind": 21428.75
    },
    "stateDatetime": "2022-07-07T10:00:00Z",
    "totalProduction": 66787.5
}
```

<h3><img src="images/logstash.png" width="18px" align="left">  &nbsp; Logstash  </h3>

Con Logstash vengono prelevati i dati, tramite il plug-in "http", che effettua richieste al server flask ogni secondo, in particolare vi sono due container Logstash per simulare più sorgenti dati e per rendere l'invio di dati più veloce.<br>
I dati vengono poi inviati ad un cluster Kafka, per proseguire il loro viaggio lungo tutta la pipeline.

<h3><img src="images/kafka.png" width="18px" align="left">  &nbsp; Zookeeper + Kafka  </h3>

Zookeeper è un software che funge da servizio centralizzato e viene utilizzato per mantenere i dati di denominazione e configurazione e per fornire una sincronizzazione flessibile e robusta all'interno dei sistemi distribuiti. <br>
Zookeeper tiene traccia dello stato dei nodi del cluster Kafka e tiene anche traccia degli argomenti Kafka, delle partizioni, ecc. <br>
Kafka, invece, è una piattaforma open source di stream processing, il progetto mira a creare una piattaforma a bassa latenza ed alta velocità per la gestione di dati in tempo reale.
Con Kafka i dati vengono passati ad un Topic, nel quale verranno conservati fin quando non saranno consumati da un processo spark opportunamente collegato.

<h3><img src="images/spark.png" width="110px" align="left"> &nbsp; </h3>
<br>
<br>


Apache Spark è un framework di elaborazione parallela open source che supporta l'elaborazione in memoria per migliorare le prestazioni delle applicazioni che analizzano Big Data. <br>
Per l'allenamento del modello è stata utilizzata la libreria "pyspark" ed in particolare la funzione "LinearRegression", ciò per effettuare una previsione sul prezzo dell'energia nella fascia oraria successiva a quella del dato corrente.<br>
A questo punto, con Spark Streaming, si elaborano i dati in stream applicando il modello ed effettuando una predizione sul dato, infine il risultato viene inviato ad Elastic Search.

<img src="images/candoit.jpeg" width="600px">


<h3><img src="images/elastic.png" width="18px" align="left">  &nbsp; Elasticsearch  </h3>

Elasticsearch è un motore di ricerca e analisi distribuito per tutti i tipi di dati, inclusi testuali, numerici, geospaziali, strutturati e non strutturati, grazie ad esso è possibile memorizzare i dati elaborati in tempo reale ed inviarli infine a Kibana per le varie visualizzazioni.




```
{
  "name" : "elasticsearch01",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "whxyytSiQiuoe-iILAcLfQ",
  "version" : {
    "number" : "8.2.2",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "9876968ef3c745186b94fdabd4483e01499224ef",
    "build_date" : "2022-05-25T15:47:06.259735307Z",
    "build_snapshot" : false,
    "lucene_version" : "9.1.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}
```




<h3><img src="images/kibana.png" width="110px" align="left">  </h3>
<br>

Kibana è una UI utilizzata per visualizzare ed analizzare i dati di Elasticsearch. <br>
Di seguito vengono riportati alcuni grafici mostrati nella dashboard. <br>



<h3><img src="images/documents_for_country.png" width="900px" align="left">  </h3>
<br>
<br>



<h3><img src="images/price_prediction.png" width="900px" align="left">  </h3>

<br>



<h3><img src="images/expensive.png" width="900px" align="left">  </h3>

<br>



<h3><img src="images/nuclear.png" width="900px" align="left">  </h3>





